In [1]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql import functions as F
from pyspark.sql import types
from pyspark.sql.window import Window
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import StandardScaler
from pyspark.ml.clustering import KMeans

In [2]:
spark = SparkSession \
    .builder \
    .appName("example") \
    .getOrCreate()
sc = spark.sparkContext
sqlc = SQLContext(sc)

In [3]:
df = sqlc.createDataFrame([
    (0, 18, 1, [1,2]),
    (1, 25, 0, []),
    (2, 40, None, []),
    (3, 36, 0, None),
    (3, 36, 1, []),
    (4, 24, 1, []),
], ['uid', 'age', 'gender', 'list'])

In [4]:
df.show()

+---+---+------+------+
|uid|age|gender|  list|
+---+---+------+------+
|  0| 18|     1|[1, 2]|
|  1| 25|     0|    []|
|  2| 40|  null|    []|
|  3| 36|     0|  null|
|  3| 36|     1|    []|
|  4| 24|     1|    []|
+---+---+------+------+



In [5]:
df.withColumn('list', F.coalesce(df['list'], F.array(F.lit(0)))).show()

+---+---+------+------+
|uid|age|gender|  list|
+---+---+------+------+
|  0| 18|     1|[1, 2]|
|  1| 25|     0|    []|
|  2| 40|  null|    []|
|  3| 36|     0|   [0]|
|  3| 36|     1|    []|
|  4| 24|     1|    []|
+---+---+------+------+



In [6]:
df.withColumn('list', F.coalesce(df['list'], F.array())).show()

Py4JJavaError: An error occurred while calling o57.showString.
: scala.MatchError: NullType (of class org.apache.spark.sql.types.NullType$)
	at org.apache.spark.sql.catalyst.expressions.Cast.castToLong(Cast.scala:310)
	at org.apache.spark.sql.catalyst.expressions.Cast.org$apache$spark$sql$catalyst$expressions$Cast$$cast(Cast.scala:516)
	at org.apache.spark.sql.catalyst.expressions.Cast.castArray(Cast.scala:455)
	at org.apache.spark.sql.catalyst.expressions.Cast.org$apache$spark$sql$catalyst$expressions$Cast$$cast(Cast.scala:519)
	at org.apache.spark.sql.catalyst.expressions.Cast.cast$lzycompute(Cast.scala:531)
	at org.apache.spark.sql.catalyst.expressions.Cast.cast(Cast.scala:531)
	at org.apache.spark.sql.catalyst.expressions.Cast.nullSafeEval(Cast.scala:533)
	at org.apache.spark.sql.catalyst.expressions.UnaryExpression.eval(Expression.scala:327)
	at org.apache.spark.sql.catalyst.optimizer.ConstantFolding$$anonfun$apply$1$$anonfun$applyOrElse$1.applyOrElse(expressions.scala:51)
	at org

In [9]:
df = df.withColumn('aa', F.array().cast('array<bigint>'))
df.show()
df.withColumn('list', F.coalesce(df['list'], df['aa'])).show()

+---+---+------+------+---+
|uid|age|gender|  list| aa|
+---+---+------+------+---+
|  0| 18|     1|[1, 2]| []|
|  1| 25|     0|    []| []|
|  2| 40|  null|    []| []|
|  3| 36|     0|  null| []|
|  3| 36|     1|    []| []|
|  4| 24|     1|    []| []|
+---+---+------+------+---+



Py4JJavaError: An error occurred while calling o353.showString.
: scala.MatchError: NullType (of class org.apache.spark.sql.types.NullType$)
	at org.apache.spark.sql.catalyst.expressions.Cast.castToLong(Cast.scala:310)
	at org.apache.spark.sql.catalyst.expressions.Cast.org$apache$spark$sql$catalyst$expressions$Cast$$cast(Cast.scala:516)
	at org.apache.spark.sql.catalyst.expressions.Cast.castArray(Cast.scala:455)
	at org.apache.spark.sql.catalyst.expressions.Cast.org$apache$spark$sql$catalyst$expressions$Cast$$cast(Cast.scala:519)
	at org.apache.spark.sql.catalyst.expressions.Cast.cast$lzycompute(Cast.scala:531)
	at org.apache.spark.sql.catalyst.expressions.Cast.cast(Cast.scala:531)
	at org.apache.spark.sql.catalyst.expressions.Cast.nullSafeEval(Cast.scala:533)
	at org.apache.spark.sql.catalyst.expressions.UnaryExpression.eval(Expression.scala:327)
	at org.apache.spark.sql.catalyst.optimizer.ConstantFolding$$anonfun$apply$1$$anonfun$applyOrElse$1.applyOrElse(expressions.scala:51)
	at or

In [ ]:
df.na.

In [24]:
array_udf = F.udf(lambda : [], types.ArrayType(types.StringType()))
# array_udf = F.udf(lambda : [])

In [25]:
df.withColumn('list', F.coalesce(df['list'], array_udf())).show()

+---+---+------+------+---+
|uid|age|gender|  list| aa|
+---+---+------+------+---+
|  0| 18|     1|[1, 2]| []|
|  1| 25|     0|    []| []|
|  2| 40|  null|    []| []|
|  3| 36|     0|    []| []|
|  3| 36|     1|    []| []|
|  4| 24|     1|    []| []|
+---+---+------+------+---+



In [26]:
df.dtypes

[('uid', 'bigint'),
 ('age', 'bigint'),
 ('gender', 'bigint'),
 ('list', 'array<bigint>'),
 ('aa', 'array<null>')]

In [29]:
df.filter(df['gender'].isNull() == False).show()

+---+---+------+------+---+
|uid|age|gender|  list| aa|
+---+---+------+------+---+
|  0| 18|     1|[1, 2]| []|
|  1| 25|     0|    []| []|
|  3| 36|     0|  null| []|
|  3| 36|     1|    []| []|
|  4| 24|     1|    []| []|
+---+---+------+------+---+



In [44]:
len_udf = F.udf(lambda x: len(x))
df.withColumn('list_len', F.udf(lambda x: len(x))('aa')).show()

+---+---+------+------+---+--------+
|uid|age|gender|  list| aa|list_len|
+---+---+------+------+---+--------+
|  0| 18|     1|[1, 2]| []|       0|
|  1| 25|     0|    []| []|       0|
|  2| 40|  null|    []| []|       0|
|  3| 36|     0|  null| []|       0|
|  3| 36|     1|    []| []|       0|
|  4| 24|     1|    []| []|       0|
+---+---+------+------+---+--------+



In [50]:
import pandas as pd
df = pd.DataFrame([[1,2,3]], columns=['a', 'b', 'c'])
df = df.reindex(columns=['a', 'b', 'c', 'e'])
df

,a,b,c,e
0,1,2,3,NaN
